Improvement of the through time factor


AR change factor card: ARxARxS


S is the variable that controls the standard deviation parameter of the gaussian noise added to the previous day's airway resistance

In [1]:
import src.models.var_builders as var_builders
import src.data.helpers as dh
import src.data.breathe_data as bd
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import src.models.helpers as mh
import src.models.cpts.helpers as cpth
import src.modelling_ar.AR_joint_sampling as model_ar
import src.modelling_ar.ar as ar

In [2]:
AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)
AR1 = mh.VariableNode("Airway resistance day 1 (%)", 0, 90, 2, prior=None)
AR2 = mh.VariableNode("Airway resistance day 2 (%)", 0, 90, 2, prior=None)
S = mh.DiscreteVariableNode("AR change factor shape", 2, 10, 2)
S.values

array([ 2,  4,  6,  8, 10])

## Insights into different distributions

In [3]:
# Each bin of AR is uniformly distributed
# P(AR2 | AR1 = [0, 2]) is the convolution of a laplace distribution with a uniform distribution


# Let's just take the midbin for now
def laplace_pdf(x, mu, b):
    return 1 / (2 * b) * np.exp(-np.abs(x - mu) / b)


def gaussian_pdf(x, mu, sigma):
    return 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)


y_laplace = laplace_pdf(AR2.midbins, AR1.midbins[10], 5)
y_laplace = y_laplace / np.sum(y_laplace)

y_laplace = laplace_pdf(AR2.midbins, AR1.midbins[10], 5)
y_laplace = y_laplace / np.sum(y_laplace)

y_gaussian = gaussian_pdf(AR2.midbins, AR1.midbins[10], 5)
y_gaussian = y_gaussian / np.sum(y_gaussian)

fig = go.Figure()
fig.add_trace(go.Scatter(x=AR2.midbins, y=y_laplace, mode="markers+lines"))
# fig.add_trace(go.Scatter(x=AR2.midbins, y=y_gaussian, mode='markers+lines'))
fig.show()

## Uniform + laplace dist convolution validatoin

In [4]:
def PDF_laplace(x, mu, s):
    return 1 / (2 * s) * np.exp(-np.abs(x - mu) / s)


def sampler_uniform_x_laplace(n, a, b, s):
    """
    X's bin follows U(a, b)
    Y follows Laplace(mu=AR1, s=s)
    """
    X_samples = np.random.uniform(a, b, n)
    for i in range(n):
        Y_samples = np.random.laplace(X_samples, s)
    return X_samples, Y_samples


n=10000
ar_down, ar_up = AR1.get_bins_arr()[20]
shape = 10
AR1_samples, AR2_samples = sampler_uniform_x_laplace(n, ar_down, ar_up, shape)

# Create plot histogram plot for AR2 samples using go figure
fig = go.Figure()
fig.add_trace(go.Histogram(x=AR2_samples, histnorm="probability density"))
# Limit the x-axis to the range of AR2
fig.update_xaxes(range=[AR2.a, AR2.b], title="Airway resistance day 2 (%)")
fig.show()

In [7]:
import scipy.integrate as integrate


def PDF_conv_uni_laplace_additive(z1, z2, y1, y2, b, abserr_tol=1e-10):
    """
    PDF of a convolution of a uniform and a gaussian distribution

    Return P(z| y1 < y < y2)
    """
    A = 1 / (y2 - y1)

    def conv_fn(y, z1, b)
        return laplace_fn(y, z1, b) * 1 / (y2 - y1)

    val, abserr = integrate.dblquad(laplace_fn, y1, y2, args=(z1, b))
    if abserr > abserr_tol:
        raise ValueError(
            f"Absolute error after solving the integral is too high {abserr}"
        )

    return A * val

In [10]:
# Compare for one bin

a, b = AR1.get_bins_arr()[15]
val = PDF_conv_uni_laplace_additive(AR2.midbins[15], a, b, 5)

In [4]:
# Build CPT

cpt = np.zeros((AR2.card, AR2.card, S.card))

for b_idx, b in enumerate(S.values):
    for i in range(AR1.card):
        ar1_down, ar1_up = AR1.get_bins_arr()[i]
        p_ar2 = ar.PDF_conv_uni_laplace_additive(z, ar1_down, ar1_up, b)
        p_ar2 = p_ar2 / np.sum(p_ar2)
        cpt[:, i, b_idx] = p_ar2

In [5]:
cpth.save_cpt([AR, AR, S], cpt, "_shape_factor")